In [1]:
#basic libraries
import os
import pandas as pd 
import numpy as np
import requests
import json
import glob
import pickle

#Visiualization and ML libraries
import sklearn
import matplotlib.pyplot as plt

from scipy.stats import norm
import statistics
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_table('https://raw.githubusercontent.com/tfs4/liar_dataset/master/train.tsv')
df.columns =['json_id', 'claim', 'object', 'topic', 'speaker', 'job_title (string)','state_info (string)','party_affiliation (string)','barely_true_counts (float32)','false_counts (float32)','half_true_counts (float32)','	mostly_true_counts (float32)','	pants_on_fire_counts (float32)','context (string)']
df2 = df[['json_id', 'claim','object']].copy()

In [3]:
df_test = df2.head(10)
print(df_test)

      json_id        claim                                             object
0  10540.json    half-true  When did the decline of coal start? It started...
1    324.json  mostly-true  Hillary Clinton agrees with John McCain "by vo...
2   1123.json        false  Health care reform legislation is likely to ma...
3   9028.json    half-true  The economic turnaround started at the end of ...
4  12465.json         true  The Chicago Bears have had more starting quart...
5   2342.json  barely-true  Jim Dunnam has not lived in the district he re...
6    153.json    half-true  I'm the only person on this stage who has work...
7   5602.json    half-true  However, it took $19.5 million in Oregon Lotte...
8   9741.json  mostly-true  Says GOP primary opponents Glenn Grothman and ...
9   7115.json  mostly-true  For the first time in history, the share of th...


In [4]:
from pipe_line import readability_computation, compress_doc,count_named_entities, VADER_score, process_text

df_test[['flesch_kincaid_reading_ease', 'compressed_size', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']] = df_test['object'].apply(process_text).apply(pd.Series)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sandrobarreshamers/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_3939/3174439415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[['flesch_kincaid_reading_ease', 'compressed_size', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']] = df_test['object'].apply(process_text).apply(pd.Series)
/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_3939/3174439415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [5]:
df_test['num_entities'] = df_test['object'].apply(count_named_entities)

/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_3939/2386422010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['num_entities'] = df_test['object'].apply(count_named_entities)


In [6]:
df_test.head()

,json_id,claim,object,flesch_kincaid_reading_ease,compressed_size,vader_neg,vader_neu,vader_pos,vader_compound,num_entities
0,10540.json,half-true,When did the decline of coal start? It started...,71.815000,11447.0,0.000,0.902,0.098,0.3612,2
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",71.781579,9087.0,0.107,0.687,0.206,0.3182,4
2,1123.json,false,Health care reform legislation is likely to ma...,53.655000,5510.0,0.000,0.606,0.394,0.7579,0
3,9028.json,half-true,The economic turnaround started at the end of ...,61.325000,4707.0,0.000,1.000,0.000,0.0000,0
4,12465.json,true,The Chicago Bears have had more starting quart...,54.096667,12203.0,0.119,0.828,0.053,-0.4601,3


In [7]:
from pipe_line import count_pos



In [8]:
fake_string = 'The earth is falt and also made of ailens and reptiles, joe biden is the devil and pizza gate is real wwatch out people, democrats are the new nazis,.'

count_pos(fake_string)


{'DET': 3,
 'NOUN': 8,
 'AUX': 4,
 'PROPN': 5,
 'CCONJ': 3,
 'ADV': 1,
 'VERB': 1,
 'ADP': 2,
 'PUNCT': 4,
 'ADJ': 2}

In [9]:
df_test['pos_counts'] = df_test['object'].apply(count_pos)


/var/folders/0q/8tgppr255xx8v40x26xhhmhr0000gn/T/ipykernel_3939/2878837005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['pos_counts'] = df_test['object'].apply(count_pos)


In [10]:
from pipe_line import values_by_label